In [21]:
import matplotlib.pyplot as plt
import pandas as pd
from statsmodels.tsa.seasonal import seasonal_decompose
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV
import numpy as np
from datetime import datetime


from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.dummy import DummyRegressor


from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

random_seed = 0

In [22]:
data = pd.read_csv("../Merging/Merged_Data.csv")

In [23]:

data['period'] = pd.to_datetime(data['period'])
data.sort_values(by='period', inplace=True)
plant_codes = data['plantCode'].unique()
data = data[data['period'] >= '2019-01-01']
pd.set_option('display.max_colwidth', None)

In [24]:
# Define features for training and testing
model_features = ['ELEVATION', 'TEMP',
       'WDSP', 'MXSPD', 'GUST', 'MAX', 'MIN', 'PRCP', 'SNDP','TEMPEXT_BASE40', 'TEMPEXT_BASE45', 'TEMPEXT_BASE50', 
       'OVER_60', 'OVER_70', 'OVER_80', 'UNDER_40', 'UNDER_30', 'UNDER_20',
       'SUM_OVER_UNDER', 'LATITUDE','LONGITUDE','Zip','plantCode']

In [25]:
def train_and_evaluate(model, model_features, data, param_grid=None, init_test_year=None):
    results = []
    
    # Exclude negative consumption values
    data = data[data['total-consumption'] > 0]

    # Convert 'period' column to datetime format
    data = data.copy()
    data['period'] = pd.to_datetime(data['period'])

    # Extract features and target variable
    X = data[model_features].fillna(0)
    y = data['total-consumption']

    # Define the years for cross-validation folds
    years = sorted(data['period'].dt.year.unique())

    # Iterate over years for cross-validation
    for i in range(len(years)):
        train_years = years[:i+1]  # Include data from previous years
        if init_test_year == None and i + 1 < len(years):
            test_year = years[i + 1]  # Test on the following year if it exists
        elif init_test_year != None:
            test_year=init_test_year
        else:
            break  # No more test years available

        # Filter the data for training and testing
        train_data = data[data['period'].dt.year.isin(train_years)]
        test_data = data[data['period'].dt.year == test_year]

        # Extract features and target variable for training
        X_train = train_data[model_features]
        y_train = train_data['total-consumption']

        # Extract features and target variable for testing
        X_test = test_data[model_features]
        y_test = test_data['total-consumption']

        # Perform hyperparameter tuning using GridSearchCV if it exists
        if param_grid is not None:
            grid_search = GridSearchCV(model, param_grid, cv=5, scoring='neg_mean_squared_error')
            grid_search.fit(X_train, y_train)

            # Get the best model from the grid search
            best_model = grid_search.best_estimator_
            best_params = grid_search.best_params_

            # Make predictions on the test set
            y_pred = best_model.predict(X_test)
        else:
            best_model=model
            best_params = None
            model.fit(X_train, y_train)

            # Make predictions on the test set
            y_pred = model.predict(X_test)

        # Evaluate the model
        mse = mean_squared_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)

        # Store results along with training years and test year
        results.append({'Mean Squared Error': mse, 'R-squared': r2, 'Training Years': train_years, 'Test Year': test_year, 'Best Parameters': best_params})

    results_df = pd.DataFrame(results)
    return results_df

In [28]:
dummy_model = DummyRegressor(strategy="mean")
train_and_evaluate(dummy_model, model_features, data)

,Mean Squared Error,R-squared,Training Years,Test Year,Best Parameters
0,2.514660e+12,-0.000019,[2019],2020,None
1,2.064796e+12,-0.015654,"[2019, 2020]",2021,None
2,2.019619e+12,-0.012861,"[2019, 2020, 2021]",2022,None
3,5.035117e+12,-0.297912,"[2019, 2020, 2021, 2022]",2023,None


In [80]:
param_grid_rf = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10]}

# Create a Random Forest model
rf_model = RandomForestRegressor(random_state = random_seed)

# Call the function with the model and other parameters
train_and_evaluate(rf_model, model_features, data, param_grid_rf)

,Mean Squared Error,R-squared,Training Years,Test Year,Best Parameters
0,4.423929e+11,0.824071,[2019],2020,"{'max_depth': 20, 'min_samples_split': 2, 'n_estimators': 100}"
1,5.175337e+11,0.745430,"[2019, 2020]",2021,"{'max_depth': 20, 'min_samples_split': 5, 'n_estimators': 50}"
2,2.164412e+11,0.891452,"[2019, 2020, 2021]",2022,"{'max_depth': 20, 'min_samples_split': 10, 'n_estimators': 150}"
3,4.967570e+11,0.871950,"[2019, 2020, 2021, 2022]",2023,"{'max_depth': 20, 'min_samples_split': 10, 'n_estimators': 150}"


In [91]:
param_grid_knn = {
    'n_neighbors': [3, 5, 7],
    'weights': ['uniform', 'distance'],
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']}

knn_model = KNeighborsRegressor()

# Call the train_test_and_plot function
train_and_evaluate(knn_model, model_features, data, param_grid_knn)


,Mean Squared Error,R-squared,Training Years,Test Year,Best Parameters
0,2.930567e+11,0.883458,[2019],2020,"{'algorithm': 'auto', 'n_neighbors': 3, 'weights': 'uniform'}"
1,2.276265e+11,0.888033,"[2019, 2020]",2021,"{'algorithm': 'ball_tree', 'n_neighbors': 7, 'weights': 'distance'}"
2,2.077719e+11,0.895800,"[2019, 2020, 2021]",2022,"{'algorithm': 'ball_tree', 'n_neighbors': 7, 'weights': 'distance'}"
3,5.066953e+11,0.869388,"[2019, 2020, 2021, 2022]",2023,"{'algorithm': 'ball_tree', 'n_neighbors': 7, 'weights': 'distance'}"


In [79]:
nb_model = GaussianNB() # Gaussian NB does not have any hyperparameters to tune, unlike the others here.
train_and_evaluate(nb_model, model_features, data)


,Mean Squared Error,R-squared,Training Years,Test Year,Best Parameters
0,7.220006e+11,0.712878,[2019],2020,None
1,5.675818e+11,0.720812,"[2019, 2020]",2021,None
2,5.087411e+11,0.744861,"[2019, 2020, 2021]",2022,None
3,1.047925e+12,0.729874,"[2019, 2020, 2021, 2022]",2023,None


In [95]:
param_grid_rf = {
    'n_estimators': [150],
    'max_depth': [20],
    'min_samples_split': [10]}

# Create a Random Forest model
rf_model = RandomForestRegressor(random_state = random_seed)

# Call the function with the model and other parameters
train_and_evaluate(rf_model, model_features, data, param_grid_rf, init_test_year=datetime(2023,1,1).year)

,Mean Squared Error,R-squared,Training Years,Test Year,Best Parameters
0,1.162417e+12,0.700362,[2019],2023,"{'max_depth': 20, 'min_samples_split': 10, 'n_estimators': 150}"
1,7.314699e+11,0.811448,"[2019, 2020]",2023,"{'max_depth': 20, 'min_samples_split': 10, 'n_estimators': 150}"
2,6.416934e+11,0.834589,"[2019, 2020, 2021]",2023,"{'max_depth': 20, 'min_samples_split': 10, 'n_estimators': 150}"
3,4.888794e+11,0.873981,"[2019, 2020, 2021, 2022]",2023,"{'max_depth': 20, 'min_samples_split': 10, 'n_estimators': 150}"
4,1.492468e+11,0.961528,"[2019, 2020, 2021, 2022, 2023]",2023,"{'max_depth': 20, 'min_samples_split': 10, 'n_estimators': 150}"


In [96]:
param_grid_knn = {
    'n_neighbors': [7],
    'weights': ['distance'],
    'algorithm': ['ball_tree']}

knn_model = KNeighborsRegressor()

# Call the train_test_and_plot function
train_and_evaluate(knn_model, model_features, data, param_grid_knn, init_test_year=datetime(2023,1,1).year)

,Mean Squared Error,R-squared,Training Years,Test Year,Best Parameters
0,9.528676e+11,0.754378,[2019],2023,"{'algorithm': 'ball_tree', 'n_neighbors': 7, 'weights': 'distance'}"
1,7.885781e+11,0.796727,"[2019, 2020]",2023,"{'algorithm': 'ball_tree', 'n_neighbors': 7, 'weights': 'distance'}"
2,6.922015e+11,0.821570,"[2019, 2020, 2021]",2023,"{'algorithm': 'ball_tree', 'n_neighbors': 7, 'weights': 'distance'}"
3,5.066953e+11,0.869388,"[2019, 2020, 2021, 2022]",2023,"{'algorithm': 'ball_tree', 'n_neighbors': 7, 'weights': 'distance'}"
4,0.000000e+00,1.000000,"[2019, 2020, 2021, 2022, 2023]",2023,"{'algorithm': 'ball_tree', 'n_neighbors': 7, 'weights': 'distance'}"


In [97]:
nb_model = GaussianNB() # Gaussian NB does not have any hyperparameters to tune, unlike the others here.
train_and_evaluate(nb_model, model_features, data, init_test_year=datetime(2023,1,1).year)

,Mean Squared Error,R-squared,Training Years,Test Year,Best Parameters
0,2.983311e+12,0.230986,[2019],2023,None
1,2.047261e+12,0.472274,"[2019, 2020]",2023,None
2,1.720580e+12,0.556483,"[2019, 2020, 2021]",2023,None
3,1.047925e+12,0.729874,"[2019, 2020, 2021, 2022]",2023,None
4,5.944781e+09,0.998468,"[2019, 2020, 2021, 2022, 2023]",2023,None


In [18]:

def feature_ablation_analysis(model, model_features, data):
    results = []

    # Filter data for training and testing years
    train_data = data[data['period'].dt.year.between(2019, 2022)]
    test_data = data[data['period'].dt.year == 2023]

    # Group 1: Only plantcode
    selected_features_group1 = ['plantCode']
    
    # Group 2: Plantcode + Location features
    selected_features_group2 = ['plantCode', 'LATITUDE', 'LONGITUDE', 'Zip']

    # Group 3: Plantcode + Weather features
    selected_features_group3 = ['plantCode']
    selected_features_group3.extend(model_features)

    # Group 4: All features together
    selected_features_group4 = ['plantCode', 'LATITUDE', 'LONGITUDE', 'Zip']
    selected_features_group4.extend(model_features)

    # Define groups
    groups = {
        "PlantCode Only": selected_features_group1,
        "PlantCode + Location": selected_features_group2,
        "PlantCode + Weather": selected_features_group3,
        "All Features Together": selected_features_group4
    }

    # Perform feature ablation analysis for each group
    for group_name, selected_features in groups.items():
        # Extract selected features and target variable for training and testing
        X_train = train_data[selected_features]
        y_train = train_data['total-consumption']
        X_test = test_data[selected_features]
        y_test = test_data['total-consumption']

        # Fit the model
        model.fit(X_train, y_train)

        # Make predictions on the test set
        y_pred = model.predict(X_test)

        # Calculate MSE and R2
        mse = mean_squared_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)

        # Store results
        results.append({'Group': group_name, 'Features Used': ', '.join(selected_features), 'Mean Squared Error': mse, 'R-squared': r2})

    # Convert results to DataFrame
    results_df = pd.DataFrame(results)
    return results_df


In [20]:
rf_model = RandomForestRegressor(random_state=random_seed,
    n_estimators = 150,
    max_depth = 20,
    min_samples_split = 10)

model_features = ['ELEVATION', 'TEMP',
       'WDSP', 'MXSPD', 'GUST', 'MAX', 'MIN', 'PRCP', 'SNDP','TEMPEXT_BASE40', 'TEMPEXT_BASE45', 'TEMPEXT_BASE50', 
       'OVER_60', 'OVER_70', 'OVER_80', 'UNDER_40', 'UNDER_30', 'UNDER_20',
       'SUM_OVER_UNDER']

feature_ablation_analysis(rf_model, model_features, data)

,Group,Features Used,Mean Squared Error,R-squared
0,PlantCode Only,plantCode,7.453344e+11,0.806544
1,PlantCode + Location,"plantCode, LATITUDE, LONGITUDE, Zip",7.323699e+11,0.809909
2,PlantCode + Weather,"plantCode, ELEVATION, TEMP, WDSP, MXSPD, GUST, MAX, MIN, PRCP, SNDP, TEMPEXT_BASE40, TEMPEXT_BASE45, TEMPEXT_BASE50, OVER_60, OVER_70, OVER_80, UNDER_40, UNDER_30, UNDER_20, SUM_OVER_UNDER",5.057196e+11,0.868737
3,All Features Together,"plantCode, LATITUDE, LONGITUDE, Zip, ELEVATION, TEMP, WDSP, MXSPD, GUST, MAX, MIN, PRCP, SNDP, TEMPEXT_BASE40, TEMPEXT_BASE45, TEMPEXT_BASE50, OVER_60, OVER_70, OVER_80, UNDER_40, UNDER_30, UNDER_20, SUM_OVER_UNDER",4.871077e+11,0.873568
